In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch

from belt_nlp.bert_with_pooling import BertClassifierWithPooling

# Example - Model BERT with pooling

In this notebook we will show how to use basic methods `fit` and `predict` for the BERT model with pooling.

In [13]:
!export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Load data - sample of IMDB reviews in english

In [14]:
raw_data_path = '../data/raw/'
processed_data_path = '../data/processed/'

In [15]:
target = 'ig'

# read data
# df = pd.read_csv(
#     raw_data_path + f'r3_{target}_test_users.csv', 
#     sep = ';', 
#     encoding='utf-8-sig'
#     )

# read data
df = pd.read_csv(
    processed_data_path + f'train_r3_{target}_top_mentioned_timelines_filtered_Texts.csv', 
    sep = ';', 
    encoding='utf-8-sig'
    )

df.Polarity = df.Polarity.map({
    "against": 0,
    "for": 1
})

In [16]:
df.head()

,User_ID,Polarity,Texts,filtered_Texts
0,r2_ig_1,0,PQP ESSE DORAMA É MUITO FOADA(Sassy GoGo(Cheer...,"@Rachingaio e iria abrir o portão, daí vejo os..."
1,r2_ig_4,1,Golaço!!!!!!!!! # Manda geral do time principa...,@lucaszeu97 Pastor Nei mandou fogo # @Paparazz...
2,r2_ig_7,0,"@gabycunha86 Amanhã vou aí, deixa pra terça # ...",NaN
3,r2_ig_8,1,3.4- O Centro de Coordenação da Operação está ...,"- Cerimônia de sanção da Lei 14.064, Lei ""SANS..."
4,r2_ig_10,1,"Me arrependi de excluir meu outro tt, agora ti...",NaN


## Divide to train and test sets

In [17]:
df['filtered_Texts'] = df.Texts.apply(lambda x: ' # '.join(x.split(' # ')[:10]))

In [18]:
texts = df["filtered_Texts"].tolist()
labels = df["Polarity"].tolist()
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

## Fit the model

In [19]:
bert_model_name = 'pablocosta/bertabaporu-base-uncased'
epochs = 1
batch_size = 1

In [22]:
MODEL_PARAMS = {
    "batch_size": 1,
    "learning_rate": 5e-5,
    "epochs": epochs,
    "chunk_size": 510,
    "stride": 510,
    "minimal_chunk_length": 510,
    "pooling_strategy": "mean",
    "pretrained_model_name_or_path": 'pablocosta/bertabaporu-base-uncased'
}
model = BertClassifierWithPooling(**MODEL_PARAMS, device="cuda:0")

OutOfMemoryError: CUDA out of memory. Tried to allocate 188.00 MiB. GPU 0 has a total capacity of 5.79 GiB of which 194.94 MiB is free. Including non-PyTorch memory, this process has 5.02 GiB memory in use. Of the allocated memory 4.61 GiB is allocated by PyTorch, and 321.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
X_train

['0 anos do gol de Basílio, o redentor de uma nação. A transcrição da narração do gênio Osmar Santos arrepia. # @felipe_1910 Inspiração pura do Osmar Santos. # @ademarjoa @brunosprado O Tite está acima do Telê na minha modesta opinião, mas sei que sou voto vencido... # Basílio, o predestinado, o escolhido... # O Corinthians ficou 0 anos, 0 meses e 0 dias sem ganhar um título paulista. Da manhã de 0 de outubro de 0 até hoje, 0 anos depois, o Corinthians venceu 0 títulos oficiais, uma média de 0 título a cada 0 ano, 0 meses e 0 dias. Doce mistério da vida este Corinthians... # O gol do Basílio em 0 é o gol mais repetido da história da TV Brasileira. # @tathiane_vidal Esse é o maior jogo da história do futebol paulista disparado. Foi a maior festa popular da história na opinião até de rivais. # @BiffaniFilho Sim. Sem dúvida. # O gol do Basílio não é só o gol mais importante da história do Corinthians. É o gol mais importante da história do futebol paulista. # Zé Duarte, técnico da Ponte P

In [ ]:
torch.cuda.empty_cache()


model.fit(X_train, y_train, epochs=epochs)  #  Warning about tokeninizing too long text is expected

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 5.79 GiB of which 60.50 MiB is free. Including non-PyTorch memory, this process has 5.04 GiB memory in use. Of the allocated memory 4.64 GiB is allocated by PyTorch, and 321.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Get predictions

In [ ]:
preds = model.predict_classes(X_test)

## Calculate model accuracy on the test data

In [ ]:
accurate = sum(preds == np.array(y_test).astype(bool))
accuracy = accurate / len(y_test)

print(f"Test accuracy: {accuracy}")

Test accuracy: 0.5611111111111111
